# Sesión 3 - Cuaderno 1: El Optimizador de Consultas y el Comando `EXPLAIN`

### **Validación del Diseño Físico**

En la Sesión 2, tomamos decisiones explícitas sobre el diseño físico de nuestras tablas en la capa Plata. Implementamos dos técnicas clave:

1.  **Particionamiento (`PARTITIONED BY`):** Organizamos los datos de las tablas de hechos en subdirectorios basados en la columna de fecha.
2.  **Z-Ordering (`ZORDER BY`):** Reorganizamos la disposición de los datos dentro de los archivos para co-localizar valores de columnas relacionados.

Estas no fueron decisiones arbitrarias. La hipótesis es que le proporcionan al motor de Spark información estructural para que pueda leer menos datos y, por lo tanto, resolver las consultas más rápidamente.

Esta sesión se centra en **verificar esa hipótesis**. Aprenderemos a usar la herramienta principal para entender cómo Spark interpreta y ejecuta nuestras consultas: el **Plan de Ejecución**.

### **El Rol del Optimizador Catalyst**

Cuando enviamos una consulta a Spark, no se ejecuta de inmediato. Primero, pasa a través de un componente llamado [**Catalyst Optimizer**](https://www.databricks.com/glossary/catalyst-optimizer). La función de Catalyst es transformar la consulta SQL (el *qué* queremos) en un procedimiento optimizado paso a paso (el *cómo* se va a ejecutar). Este procedimiento se conoce como el **Plan de Ejecución**.

El plan tiene varias fases, pero nos centraremos en la más importante para el análisis de rendimiento: el **Plan Físico**. Este es el plan final que describe las operaciones exactas que Spark realizará, incluyendo cómo accederá a los archivos, qué algoritmos de `JOIN` usará y en qué orden aplicará los filtros.

No obstante estudiemos en detalle cada fase:
-----

Cuando Spark recibe una consulta, inicia un sofisticado proceso de transformación a través de su optimizador, **Catalyst**. El objetivo es convertir el código declarativo (SQL) en un plan de ejecución optimizado para su distribución en el clúster.

A continuación, se desglosa cada componente y etapa del diagrama:

![plan ejecucion](plan_ejecucion.png)

-----
#### **Consulta SQL / DataFrame**

  * **Qué es:** Es el punto de partida de todo el proceso. Puede ser una cadena de texto con código SQL (ej. `SELECT * FROM ...`) o una secuencia de transformaciones sobre un DataFrame en Python, Scala o R (ej. `df.filter(...).join(...)`). A nivel conceptual, ambos representan la misma intención del usuario.

-----

#### **Plan Lógico sin Resolver**

  * **Qué es:** La primera representación interna y abstracta de la consulta. Catalyst "parsea" (analiza sintácticamente) el código y lo convierte en un árbol de operadores lógicos.
  * **Tecnología:** Este árbol es una estructura de datos que representa las operaciones solicitadas (proyección, filtro, unión) y las relaciones entre ellas.
  * **Estado:** Se denomina "sin resolver" porque en esta etapa **no se ha verificado nada contra los metadatos del sistema**. Spark ha validado que la sintaxis es correcta, pero no sabe si la tabla `usuarios_silver` o la columna `ciudad` realmente existen, ni conoce sus tipos de datos.

-----

#### **Analizador y Catálogo/Metastore**

  * **Qué es el Analizador:** Es el componente encargado de validar y resolver el plan lógico. Toma el árbol "sin resolver" y lo contrasta con la información disponible en el **Catálogo**.
  * **Qué es el Catálogo/Metastore:** Es el sistema central que almacena todos los metadatos sobre los activos de datos.
      * **Tecnología:** En Databricks, esta función la cumple **Unity Catalog** (la solución de gobernanza moderna y centralizada) o, en entornos más tradicionales, el **Hive Metastore**. Este catálogo contiene información vital: nombres de bases de datos, tablas, vistas, columnas, sus tipos de datos y la ubicación de los archivos físicos.
  * **Proceso:** El Analizador utiliza el Catálogo para verificar que todas las tablas y columnas referenciadas en la consulta existan. Si algo no se encuentra, el proceso falla aquí mismo con un `AnalysisException` (el típico error de "tabla o vista no encontrada").

-----

#### **Plan Lógico Analizado**

  * **Qué es:** El resultado del paso anterior. Es un plan lógico completo, validado y semánticamente correcto.
  * **Estado:** En este punto, todas las referencias han sido "resueltas". Por ejemplo, una expresión como `SELECT *` se ha expandido a la lista completa de columnas de la tabla, y cada columna tiene asociado su tipo de dato correcto (String, Integer, Timestamp, etc.).

-----

#### **Optimizador Lógico y sus Reglas**

  * **Qué es:** Una vez que Spark tiene un plan lógico válido, este componente aplica un conjunto de **reglas de optimización** para reescribirlo en una forma lógicamente equivalente, pero computacionalmente más eficiente.
  * **Proceso:** Aplica estas reglas de forma iterativa hasta que el plan ya no puede ser mejorado.
  * **Tecnología (Reglas de Optimización):** Son transformaciones de árboles predefinidas. Las más importantes son:
      * **Predicate Pushdown (Empuje de Predicados):** La regla más importante. Mueve los filtros (`WHERE`, `HAVING`) tan cerca de la fuente de datos como sea posible en el plan. Esto reduce drásticamente la cantidad de datos que necesitan ser leídos y procesados en las etapas posteriores.
      * **Projection Pruning (Poda de Proyecciones):** Elimina del plan cualquier columna que no sea estrictamente necesaria para el resultado final de la consulta o para operaciones intermedias (como una clave de `JOIN`). Esto minimiza el I/O al leer los datos de los archivos.
      * **Constant Folding (Plegado de Constantes):** Resuelve expresiones que son constantes en tiempo de planificación. `WHERE ventas > 100 * 2` se convierte internamente en `WHERE ventas > 200`, evitando cálculos repetitivos en cada fila.

-----

#### **Plan Lógico Optimizado**

  * **Qué es:** El resultado de aplicar todas las reglas de optimización. Representa el plan lógico más eficiente posible, pero sigue siendo una descripción abstracta de la consulta, no un plan ejecutable.

-----

#### **Planificador Físico y sus Estrategias**

  * **Qué es:** Este es el puente crucial entre el mundo lógico (el "qué") y el físico (el "cómo"). Su trabajo es tomar el plan lógico optimizado y generar uno o más **Planes Físicos** posibles.
  * **Proceso:** Traduce los operadores lógicos abstractos en operadores físicos ejecutables, que son implementaciones concretas.
  * **Tecnología (Estrategias):** Para cada operador lógico, pueden existir varias implementaciones físicas o "estrategias". Por ejemplo, para un `Join` lógico, el planificador podría proponer:
      * **`BroadcastHashJoin`**: Una estrategia muy eficiente si una de las tablas es lo suficientemente pequeña como para ser copiada (broadcast) a todos los nodos ejecutores.
      * **`SortMergeJoin`**: Una estrategia robusta para unir dos tablas grandes, que implica ordenar los datos en ambos lados por la clave de unión y luego fusionarlos.

-----

#### **Múltiples Planes Físicos y Modelo de Costos**

  * **Qué son los Múltiples Planes Físicos:** El planificador genera todas las combinaciones válidas de estrategias, creando un conjunto de posibles planes de ejecución.
  * **Qué es el Modelo de Costos:** Una vez que hay múltiples opciones, Spark necesita decidir cuál es la mejor. Para ello, utiliza un modelo de costos que asigna una puntuación (un "costo") a cada plan. Este costo es una estimación basada en estadísticas de los datos (como tamaño de la tabla y cardinalidad de las columnas) y heurísticas sobre el costo de las operaciones (I/O, CPU, transferencia de red).

-----

#### **Plan Físico Seleccionado**

  * **Qué es:** El plan físico con el costo estimado más bajo. **Este es el plan definitivo que `EXPLAIN` nos muestra** y el que Spark ejecutará.

-----

#### **Generación de Código (Whole-Stage CodeGen)**

  * **Qué es:** La optimización final y una de las más potentes de Spark. En lugar de que el motor interprete cada operador del plan físico fila por fila (lo que introduce una sobrecarga computacional), Spark fusiona múltiples operadores en una única función optimizada.
  * **Tecnología (Tungsten Engine):** El motor de ejecución de Spark, **Tungsten**, compila dinámicamente este código de etapa en **bytecode de Java**. Esto elimina la sobrecarga de las llamadas a funciones virtuales y permite al compilador de Java (JIT) aplicar optimizaciones de bajo nivel, como mantener los datos en los registros de la CPU para un acceso ultrarrápido.

-----

#### **Código RDD Ejecutable**

  * **Qué es:** El producto final del proceso de optimización.
  * **Tecnología (RDD - Resilient Distributed Datasets):** Aunque a alto nivel trabajamos con la API de DataFrames/SQL, internamente, el código generado opera sobre la abstracción de más bajo nivel de Spark: los RDDs. El resultado final es un grafo acíclico dirigido (DAG) de RDDs que representa el trabajo a ejecutar. Este código, altamente optimizado, se serializa y se envía a los nodos ejecutores del clúster para procesar los datos de manera distribuida.

In [0]:
# Celda de configuración
# Establecemos el contexto de la base de datos para la sesión actual.

db_name = "curso_arquitecturas"
spark.sql(f"USE {db_name}")

print(f"Contexto de base de datos activo: '{db_name}'")

### **Análisis de Consultas con `EXPLAIN`: Visualizando el Resultado Final**

Hemos detallado el complejo proceso que sigue el optimizador **Catalyst**: desde un **Plan Lógico sin Resolver** hasta un **Código RDD Ejecutable**. Pasa por múltiples fases de análisis, optimización lógica y planificación física.

Ahora, la pregunta fundamental es: ¿cómo podemos, como ingenieros de datos, inspeccionar el resultado de todo este trabajo de optimización?

La respuesta es el comando `EXPLAIN`.

Cuando ejecutamos `EXPLAIN`, le pedimos a Spark que nos muestre el **Plan Físico Seleccionado**. No vemos todos los pasos intermedios (como el plan sin resolver o los múltiples planes físicos que se descartaron), sino que vemos el **plan ganador**: la estrategia final y de menor costo que Catalyst ha decidido ejecutar.

En resumen, `EXPLAIN` es nuestra ventana directa al producto final del optimizador. Es una herramienta de diagnóstico, no de procesamiento; su función es devolver este plan **sin ejecutar la consulta**.

Comenzaremos analizando una consulta simple sobre la tabla `usuarios_silver` para familiarizarnos con la estructura de este **Plan Físico Seleccionado**.

In [0]:
%sql
-- Solicitamos el plan de ejecución para una consulta de filtrado y proyección.

EXPLAIN SELECT nombre_usuario, fecha_registro
FROM usuarios_silver
WHERE ciudad = 'Bogotá D.C.';

### **Análisis del Resultado del Plan Físico (Motor Photon)**

Este resultado nos muestra un plan de ejecución que ha sido completamente acelerado por **Photon**, el motor de ejecución nativo de alto rendimiento de Databricks, escrito en C++. La presencia de los operadores `Photon...` confirma que no estamos usando el motor estándar de Spark, sino su contraparte más rápida.

Recordemos que el plan físico se lee **de abajo hacia arriba**.

---

#### `== Physical Plan ==`

* **`PhotonScan parquet ...`**
    Este es el punto de partida y la operación más importante: la lectura de los datos.
    * **`PhotonScan parquet workspace.curso_arquitecturas.usuarios_silver`**: Indica que Photon está escaneando la tabla `usuarios_silver`, que está almacenada en formato **Parquet**.
    * **`DataFilters: [..., (ciudad#10113 = Bogotá D.C.)]`**: ¡Esta es la evidencia clave del **Predicate Pushdown**! El filtro `WHERE ciudad = 'Bogotá D.C.'` no se aplica después de cargar los datos; se "empuja" directamente a la operación de escaneo. Photon leerá los archivos y descartará las filas que no cumplan esta condición al nivel más bajo posible, minimizando drásticamente el movimiento de datos.
    * **`ReadSchema: struct<nombre_usuario:string,fecha_registro:timestamp,ciudad:string>`**: Esto confirma la **Poda de Proyecciones (Column Pruning)**. Observa que, aunque tu `SELECT` solo pedía `nombre_usuario` y `fecha_registro`, el `Scan` también lee la columna `ciudad`. Esto es porque la necesita para aplicar el `DataFilter`. Sin embargo, ignora por completo otras columnas de la tabla (como `id_usuario` y `email`), demostrando que solo lee del disco la información estrictamente necesaria.
    * **`PartitionFilters: []`**: Esta línea es muy importante. Nos dice que no se están aplicando filtros de partición. Esto es correcto y esperado, ya que la tabla `usuarios_silver` no fue particionada. En el próximo cuaderno, veremos cómo esta sección se vuelve fundamental cuando analicemos la tabla `pedidos_silver`.

* **`PhotonProject [...]`**
    Este operador corresponde a la cláusula `SELECT` de tu consulta. Una vez que `PhotonScan` ha leído y filtrado las filas, `PhotonProject` toma esas filas y selecciona únicamente las columnas que pediste en el resultado final: `nombre_usuario` y `fecha_registro`. La columna `ciudad`, que fue necesaria para el filtro, ya no es necesaria y se descarta en este punto.

* **`PhotonResultStage`**
    Es un operador interno de Photon que marca el final de una etapa de cálculo dentro del motor.

* **`ColumnarToRow`**
    Esta es una operación de transición técnica. Photon procesa datos en un formato **columnar** (los valores de una misma columna se agrupan en memoria), lo cual es extremadamente eficiente. El motor original de Spark trabaja en un formato basado en **filas**. Este operador simplemente convierte el resultado final del formato columnar de Photon al formato de fila que el resto del sistema Spark espera recibir.

---

#### `== Photon Explanation ==`

* **`The query is fully supported by Photon.`**
    Este es un mensaje de confirmación muy positivo. Significa que toda la consulta, de principio a fin, pudo ser ejecutada dentro del motor nativo y optimizado de Photon, sin necesidad de "caer" al motor estándar de Spark para ninguna operación. Esto garantiza el máximo rendimiento posible.

---

#### `== Optimizer Statistics ==`

* **`full = usuarios_silver`**
    Esta línea nos informa que el optimizador Catalyst tenía **estadísticas completas y actualizadas** sobre la tabla `usuarios_silver` al momento de crear el plan. Estas estadísticas (que incluyen el número de filas, tamaño de la tabla, distribución de los datos, etc.) son vitales para que el **Modelo de Costos** pueda tomar la decisión más inteligente al elegir entre diferentes planes físicos. Un estado `full` es ideal.

### **Prueba de la Optimización Lógica**

Además de las optimizaciones de acceso a datos, Catalyst también es capaz de razonar sobre la lógica de la consulta para simplificar o evitar trabajo.

Analizaremos el plan para dos consultas lógicamente deterministas:
1.  Una consulta con una condición que es inherentemente falsa (`WHERE 1 = 0`).
2.  Una consulta con una condición que es inherentemente verdadera y, por lo tanto, redundante (`WHERE 1 = 1`).

In [0]:
%sql
-- Caso 1: Condición lógicamente falsa
EXPLAIN SELECT * FROM usuarios_silver WHERE 1 = 0;

Este plan de ejecución es la evidencia de que el optimizador de Spark (Catalyst) ha determinado que la consulta **no necesita acceder a ningún dato** para saber que el resultado estará vacío. Es el plan más eficiente posible porque evita por completo la costosa operación de lectura de archivos (I/O).

---

#### `== Physical Plan ==`

* **`LocalTableScan <empty>, [...]`**
    * **`LocalTableScan`**: Este es el operador clave. A diferencia de `PhotonScan` o `FileScan`, que leen datos de un almacenamiento distribuido (como S3), `LocalTableScan` es una operación que se ejecuta localmente en el nodo "driver" de Spark. Se utiliza cuando Spark puede generar el resultado de la consulta sin tener que leer datos de los nodos "workers".
    * **`<empty>`**: Esta es la parte más importante. Indica que el resultado de esta operación es una tabla **vacía**. El optimizador Catalyst analizó la lógica de tu consulta (probablemente una condición `WHERE` que es lógicamente imposible, como `WHERE 1 = 0` o `WHERE ciudad = 'A' AND ciudad = 'B'`) y concluyó que ninguna fila podría cumplirla jamás.
    * **`[...]`**: La lista de columnas (`id_usuario#10158L`, etc.) simplemente define el esquema (la estructura de las columnas) que el resultado *tendría* si no estuviera vacío.

En resumen, el plan físico consiste en un solo paso: generar una tabla vacía con la estructura correcta, directamente en el driver, sin contactar a los workers ni leer un solo byte de la tabla `usuarios_silver`.

---

#### `== Photon Explanation ==`

* **`Photon does not fully support the query because: Unsupported node: LocalTableScan`**
    * Este mensaje nos informa por qué el motor Photon no se utilizó. Photon está diseñado para acelerar el procesamiento de grandes volúmenes de datos de forma distribuida y columnar.
    * La operación `LocalTableScan` es una operación muy simple, no distribuida y que no procesa datos. Por lo tanto, no hay nada que Photon pueda acelerar. Sería ineficiente invocar un motor de alto rendimiento para una tarea tan trivial.
    * En consecuencia, Spark ejecuta esta operación utilizando su motor estándar, ya que es la herramienta adecuada para este trabajo específico.

### **Clave**

Aunque a primera vista podría parecer un "error" que Photon no se use, este plan es en realidad una **demostración de la inteligencia del optimizador**. Es el resultado de una optimización lógica exitosa que ahorra el 100% del trabajo de lectura de datos, resultando en una ejecución casi instantánea.

In [0]:
%sql
-- Caso 2: Condición lógicamente redundante
EXPLAIN SELECT * FROM usuarios_silver WHERE 1 = 1;

Este plan de ejecución corresponde a una consulta que solicita todos los datos de la tabla `usuarios_silver`, como un `SELECT * FROM usuarios_silver;` sin ninguna cláusula `WHERE`. Es el ejemplo perfecto de un **escaneo completo de tabla (Full Table Scan)**.

Vamos a desglosar cada componente para entender por qué es así.

---

#### `== Physical Plan ==`

* **`PhotonScan parquet ...`**
    Esta es la operación de lectura de datos.
    * **`PhotonScan parquet workspace.curso_arquitecturas.usuarios_silver`**: De nuevo, Photon está escaneando la tabla `usuarios_silver` en su formato Parquet subyacente.
    * **`DataFilters: []`** y **`RequiredDataFilters: []`**: Esta es la diferencia fundamental con el primer plan que analizamos. Ambas listas están **vacías**. Esto confirma que no hay una cláusula `WHERE` en la consulta, por lo que **no se está aplicando ningún filtro de predicado**. Photon no puede descartar ninguna fila durante el escaneo y debe leer todo el contenido de los archivos.
    * **`PartitionFilters: []`**: Al igual que antes, esta lista está vacía, lo cual es correcto porque la tabla `usuarios_silver` no está particionada.
    * **`ReadSchema: struct<id_usuario:bigint, ...>`**: El esquema de lectura incluye **todas las columnas** de la tabla. Esto confirma que no hay **Poda de Proyecciones (Column Pruning)**, lo cual es lógico para una consulta `SELECT *`. Spark debe leer todas las columnas del disco porque todas fueron solicitadas.

* **`PhotonResultStage` y `ColumnarToRow`**
    Estos operadores cumplen la misma función que en el primer ejemplo. `PhotonResultStage` marca el final de la computación en Photon, y `ColumnarToRow` convierte los datos del formato columnar optimizado de Photon al formato de fila estándar de Spark para devolver el resultado final.

---

#### `== Photon Explanation ==`

* **`The query is fully supported by Photon.`**
    Este mensaje indica que, a pesar de ser un escaneo completo, toda la operación de lectura y procesamiento de datos fue manejada por el motor nativo de Photon, lo cual asegura que se realizó de la manera más rápida posible.

---

#### `== Optimizer Statistics ==`

* **`full = usuarios_silver`**
    Nuevamente, esto confirma que el optimizador tenía estadísticas completas de la tabla para planificar la consulta. Aunque en una consulta tan simple como un escaneo completo no hay muchas decisiones que tomar, tener estadísticas completas sigue siendo una buena práctica para la salud general del sistema.

### **Clave**

Este plan de ejecución es la representación de la operación de lectura más básica y, a menudo, la más costosa: un escaneo completo de tabla. Nos muestra lo que sucede cuando no se proporcionan filtros: **Spark debe leer todas las columnas y todas las filas de la tabla**. Es el comportamiento esperado para una consulta sin filtros, y `EXPLAIN` nos permite verificarlo con total certeza.

### **Ampliando el Análisis: El Plan de Ejecución de un `JOIN`**

Las consultas en un Data Warehouse raramente involucran una sola tabla. La operación fundamental que conecta nuestras tablas de hechos con las dimensiones es el `JOIN`. Dado que los `JOINs` pueden ser computacionalmente intensivos, entender cómo Spark planea ejecutarlos es crucial para el rendimiento.

Antes de analizar un plan de `JOIN`, primero debemos entender las estrategias que Spark puede emplear.

-----

### **Estrategias de JOIN en Profundidad**

Un `JOIN` requiere mover y comparar datos de múltiples tablas a través de la red del clúster. Spark tiene principalmente dos estrategias para resolver esta costosa operación.

#### Estrategia 1: Broadcast Hash Join (BHJ) - "La Copia Rápida"

Esta es, por lejos, la estrategia de `JOIN` más rápida y eficiente, pero solo se puede aplicar cuando una de las tablas en el `JOIN` es **significativamente más pequeña** que la otra.

  * **Concepto:**
    1.  **Broadcast:** La tabla pequeña se recolecta por completo en el nodo Driver y luego es copiada a cada uno de los nodos Ejecutores.
    2.  **Tabla Hash:** Cada Ejecutor recibe la copia y la carga en su memoria en una tabla hash para búsquedas instantáneas.
    3.  **Unión Local:** La tabla grande no se mueve. Los Ejecutores leen sus particiones de la tabla grande y, para cada fila, buscan la clave de unión en la tabla hash local.

Como la unión ocurre localmente sin mover la tabla grande, **se evita un costoso Shuffle**.

  * ✅ **Ventajas:** Extremadamente rápido al eliminar el Shuffle de la tabla grande.
  * ❌ **Desventajas:** Limitado por el tamaño. Si la tabla pequeña no cabe en la memoria del Driver y de cada Ejecutor, la consulta fallará. El umbral por defecto es de **10 MB**.

#### Estrategia 2: Shuffle Sort Merge Join (SMJ) - "El Plan Robusto"

Esta es la estrategia de `JOIN` más robusta y escalable, capaz de unir tablas de cualquier tamaño.

  * **Concepto:**

    1.  **Shuffle (Reorganizar):** Es la fase de redistribución. Spark lee ambas tablas y las re-particiona a través de la red, asegurando que todas las filas con la misma clave de `JOIN` de ambas tablas terminen en el mismo Ejecutor.
    2.  **Sort (Ordenar):** Dentro de cada Ejecutor, los datos recibidos de ambas tablas se ordenan por la clave del `JOIN`.
    3.  **Merge (Fusionar):** Una vez ordenados, el motor une las filas recorriendo ambas listas de datos de manera eficiente, como al cerrar una cremallera.

  * ✅ **Ventajas:** Es la estrategia más fiable y escalable, capaz de manejar tablas de terabytes.

  * ❌ **Desventajas:** El **Shuffle** es muy costoso. Implica un uso intensivo de la red y del disco.

-----

### **El Rol de Adaptive Query Execution (AQE)**

En las versiones modernas de Spark, el optimizador no se detiene en el plan inicial. **Adaptive Query Execution (AQE)** es el director de orquesta inteligente que puede **cambiar la estrategia a mitad de la ejecución**.

Por ejemplo, si AQE se da cuenta de que una tabla que inicialmente parecía grande es en realidad pequeña (quizás por un filtro muy efectivo), puede cambiar dinámicamente un `SortMergeJoin` planeado a un `BroadcastHashJoin` mucho más rápido.

Ahora que conocemos la teoría, analicemos un plan de `JOIN` real.


In [0]:
%sql
EXPLAIN SELECT
  u.nombre_usuario,
  p.monto_total
FROM pedidos_silver p
JOIN usuarios_silver u
  ON p.id_usuario = u.id_usuario
LIMIT 10;

### **Análisis del Resultado (Plan Adaptativo)**

La primera línea del resultado, `AdaptiveSparkPlan`, nos indica que la **Ejecución Adaptativa de Consultas (AQE)** está activa. Esto significa que el plan que vemos es la **hipótesis inicial** de Spark, la cual puede cambiar dinámicamente durante la ejecución si los datos reales difieren de las estimaciones.

#### `== Physical Plan ==`
* **`AdaptiveSparkPlan`**: Es el contenedor de la ejecución adaptativa. El argumento `isFinalPlan=false` confirma que este es el plan *antes* de la ejecución.
* **`== Initial Plan ==`**: Este es el plan que Catalyst generó basándose en las estadísticas. Leyéndolo de abajo hacia arriba:
    1.  **(2) PhotonScan ...usuarios\_silver**: Spark comienza a leer la tabla de dimensión `usuarios_silver`.
    2.  **(3) PhotonShuffleExchangeSink**: Este paso prepara la tabla `usuarios_silver` para ser enviada a través de la red para el broadcast.
    3.  **(6) PhotonBroadcastHashJoin**: El plan inicial es realizar un **`BroadcastHashJoin`**. La lógica del optimizador es: "Basado en las estadísticas, estimo que `usuarios_silver` es lo suficientemente pequeña como para hacer un broadcast. Esta es la estrategia más rápida".
    4.  **(1) PhotonScan ...pedidos\_silver**: Simultáneamente, Spark escanea la tabla de hechos `pedidos_silver`. La unión se realizará a medida que se leen las filas de esta tabla grande, buscándolas en la tabla hash de `usuarios_silver` que ya está en la memoria de cada ejecutor.

#### Conclusión Clave
El plan inicial nos muestra la estrategia más optimista: un **`BroadcastHashJoin`**. Gracias a que AQE está activo, si durante la ejecución se descubre que la tabla `usuarios_silver` es más grande de lo estimado, Spark tiene la capacidad de cambiar dinámicamente a un `SortMergeJoin` más robusto, asegurando el mejor rendimiento posible.

In [0]:
%sql
EXPLAIN FORMATTED SELECT
  u.nombre_usuario,
  p.monto_total
FROM pedidos_silver p
JOIN usuarios_silver u
  ON p.id_usuario = u.id_usuario
LIMIT 10;

### **Análisis del Plan Lógico Optimizado**

Al ejecutar el comando anterior, obtendrás un resultado con cuatro secciones. Nos centraremos en el **`== Optimized Logical Plan ==`**:

```

\== Optimized Logical Plan ==
GlobalLimit 10
\+- Project [nombre\_usuario\#..., monto\_total\#...]
\+- Join Inner, (id\_usuario\#...L = id\_usuario\#...L)
:- Project [id\_usuario\#...L, monto\_total\#...]
:  +- Relation ... `pedidos_silver` ...
\+- Project [id\_usuario\#...L, nombre\_usuario\#...]
\+- Relation ... `usuarios_silver` ...

```

Este plan nos muestra el resultado final de todas las reglas de optimización de Catalyst. Observa las diferencias clave con el plan físico:

* **Abstracción:** Los operadores son genéricos: `Join`, `Project`, `GlobalLimit`. No hay mención de `BroadcastHashJoin` o `FileScan`. En esta fase, Spark aún no ha decidido *cómo* ejecutará la unión.
* **Confirmación de Optimizaciones:** Las operaciones de `Project` que aparecen *debajo* del `Join` son la prueba de la **Poda de Proyecciones (Projection Pruning)**. El optimizador ha reordenado el plan para asegurarse de que solo las columnas estrictamente necesarias se carguen de cada tabla antes de la costosa operación de `JOIN`.

Con esto, hemos completado el ciclo: entendemos la teoría de las fases de optimización y las estrategias de `JOIN`, y ahora podemos usar `EXPLAIN` para ver en la práctica tanto el plan abstracto (el "qué") como el plan de ejecución concreto (el "cómo").
```
